# Search-Based Software Engineering Exercise
## Exercise 02 - Genetic algorithms
<center>
    
    Stefan Jahns
    
    Softwaresysteme - Summer Term 2024
    
</center>
<center>
    <img src='uni-leipzig.png' style="height:5em" />   <img src='SOSY-Logo.png' style="height:5em" />
</center>

Goal of this exercise is to learn and understand genetic algorithms by finding an optimal configuration for highly configurable software systems

## Problem set description for this exercise

Highly configurable software systems are systems which provide many features to be selected for creating an individual variant of a program.

However, selecting and deselecting specific features for a configuration can result in efficient or an inefficient configurations of the program.

For the exercise, we want to find an optimal configuration for the *h264* and *bdbc* systems.

#### h264

h264 is a block-oriented motion-compensation-based video compression standard ([source](https://en.wikipedia.org/wiki/H.264/MPEG-4_AVC)). 

#### bdbc
Berkeley DB (bdb) is a software library intended to provide a high-performance embedded database for key/value data ([source](https://en.wikipedia.org/wiki/Berkeley_DB)). The given dataset is based on the c-implementation of bdb.

## Problem sets
- Each problem set consists of 2 files: features & interactions

### Feature file definition
- each line represents a feature of the software system
- the value at the end of the line represents the performance value of the respective feature if enabled
- for example "have_crypto: 0.0120768614910988" means that a version of the program can have a cryptographic function enabled and the feature influences the performance by 0.0120768614910988

### Interaction file definition
- each line represents a specific feature combination
- a line consists of at least 2 features separated by a "#" and a performance value for this specific selection
- for example: "CS32MB#DIAGNOSTIC: 21.0214021572657" means if CS32MB and DIAGNOSTIC are enabled, this affects the configuration performance by 21.0214021572657

In [4]:
bdbc_f = "datasets/bdbc_feature.txt"
bdbc_i = "datasets/bdbc_interactions.txt"
x264_f = "datasets/x264_feature.txt"
x264_i = "datasets/x264_interactions.txt"

#path = bdbc_f
path = bdbc_i
#path = h264_f
#path = h264_i

with open(path) as f:
    content = f.read()
print(content)

CS32MB#DIAGNOSTIC: 21.0214021572657
HAVE_HASH#DIAGNOSTIC: 30.5541602784387
HAVE_SEQUENCE#PS1K: -0.153274729661482
PS1K#HAVE_VERIFY: -0.177890062417835
HAVE_STATISTICS#HAVE_HASH: 28.0708701735602
CS32MB#HAVE_SEQUENCE: 28.3911884514065
HAVE_SEQUENCE#HAVE_CRYPTO: -0.710074228206047
PS4K#HAVE_HASH: -2.10428937899935
HAVE_VERIFY#HAVE_HASH: -2.71924279573203
PS16K#CS512MB: -4.05239323580753
PS1K#HAVE_REPLICATION: -0.732537201572133
HAVE_CRYPTO#PS8K: -0.833090135903526
HAVE_REPLICATION#PS16K#DIAGNOSTIC: 31.7326205862695
CS64MB#PS16K#DIAGNOSTIC: 8.49628382727797
PS8K#CS512MB#HAVE_HASH: -2.50486577199372
PS1K#PS4K: -10
PS1K#PS8K: -10
PS1K#PS16K: -10
PS1K#PS32K: -10
PS4K#PS8K: -10
PS4K#PS16K: -10
PS4K#PS32K: -10
PS8K#PS16K: -10
PS8K#PS32K: -10
PS16K#PS32K: -10
CS16MB#CS32MB: -10
CS16MB#CS64MB: -10
CS16MB#CS512MB: -10
CS32MB#CS64MB: -10
CS32MB#CS512MB: -10
CS64MB#CS512MB: -10



In [5]:
def readTXT(path):
    result = []
    with open(path) as f:
        lines = f.readlines()
    for line in lines:
        name = line.split(" ")[0][:-1]
        names = name.split("#")
        value = float(line.split(" ")[1].strip())
        configuration = [names, value]
        result.append(configuration)
    return result

In [6]:
readTXT(bdbc_i)

[[['CS32MB', 'DIAGNOSTIC'], 21.0214021572657],
 [['HAVE_HASH', 'DIAGNOSTIC'], 30.5541602784387],
 [['HAVE_SEQUENCE', 'PS1K'], -0.153274729661482],
 [['PS1K', 'HAVE_VERIFY'], -0.177890062417835],
 [['HAVE_STATISTICS', 'HAVE_HASH'], 28.0708701735602],
 [['CS32MB', 'HAVE_SEQUENCE'], 28.3911884514065],
 [['HAVE_SEQUENCE', 'HAVE_CRYPTO'], -0.710074228206047],
 [['PS4K', 'HAVE_HASH'], -2.10428937899935],
 [['HAVE_VERIFY', 'HAVE_HASH'], -2.71924279573203],
 [['PS16K', 'CS512MB'], -4.05239323580753],
 [['PS1K', 'HAVE_REPLICATION'], -0.732537201572133],
 [['HAVE_CRYPTO', 'PS8K'], -0.833090135903526],
 [['HAVE_REPLICATION', 'PS16K', 'DIAGNOSTIC'], 31.7326205862695],
 [['CS64MB', 'PS16K', 'DIAGNOSTIC'], 8.49628382727797],
 [['PS8K', 'CS512MB', 'HAVE_HASH'], -2.50486577199372],
 [['PS1K', 'PS4K'], -10.0],
 [['PS1K', 'PS8K'], -10.0],
 [['PS1K', 'PS16K'], -10.0],
 [['PS1K', 'PS32K'], -10.0],
 [['PS4K', 'PS8K'], -10.0],
 [['PS4K', 'PS16K'], -10.0],
 [['PS4K', 'PS32K'], -10.0],
 [['PS8K', 'PS16K'], -1

### Aim of this exercise: find an optimal configuration (= min performance value)

### Create a fitness function
- Build a fitness assessment function using the given files

feature file:

root: 4.29699011265453

HAVE_CRYPTO: 0.0120768614910988

HAVE_HASH: 4.62346632789263

HAVE_REPLICATION: 0.274902720516604

...

---

interaction file:

PS16K#CS512MB: -4.05239323580753

PS1K#HAVE_REPLICATION: -0.732537201572133

HAVE_CRYPTO#PS8K: -0.833090135903526

HAVE_REPLICATION#PS16K#DIAGNOSTIC: 31.7326205862695

...




fitness function:

4.29 + HAVE_CRYPTO * 0.01 + HAVE_HASH * 4.63 + HAVE_REPLICATION * 0.27 + ... +

PS16K * CS512MB * (-4.05) + PS1K * HAVE_REPLICATION * (- .73) + ...

## write your genetic algorithm

How to build a genetic algorithm? See lecture notes & coding session.

How to model an individual candidate solution?
- Use a feature vector with the length of all features
- represent the selection of a feature as either 0 (deselected) or 1 (selected)

For example when selecting only the feature "root" in bdbc:

[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

The fitness of this candidate solution is: 4.29699011265453